In [1]:
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense, Dropout
from sklearn.model_selection import train_test_split
import ipywidgets as widgets
from IPython.display import display, clear_output   # <— added clear_output for the new widget
import re
import random
import os
from datetime import datetime

# -------------------------------------------------------------------
# Suppress verbose TensorFlow retracing logs
# -------------------------------------------------------------------
tf.get_logger().setLevel('ERROR')

# ===============================
# Load and preprocess the data
# ===============================
df = pd.read_csv('/content/sentiment.csv')

def clean_text(text):
    text = text.lower()
    text = re.sub(r"i'm", "i am", text)
    text = re.sub(r"\'s", " is", text)
    text = re.sub(r"\'d", " would", text)
    text = re.sub(r"\'ll", " will", text)
    text = re.sub(r"\'re", " are", text)
    text = re.sub(r"\'ve", " have", text)
    text = re.sub(r"won't", "will not", text)
    text = re.sub(r"can't", "cannot", text)
    text = re.sub(r"[-()\"#/@;:<>{}+=~|.?,]", "", text)
    return text

df['cleaned_text'] = df['text'].apply(clean_text)

# Tokenization and padding
tokenizer = Tokenizer(num_words=5000, oov_token='<OOV>')
tokenizer.fit_on_texts(df['cleaned_text'])
sequences = tokenizer.texts_to_sequences(df['cleaned_text'])
padded_sequences = pad_sequences(sequences, maxlen=100, truncating='post')

# Prepare labels
labels = tf.keras.utils.to_categorical(df['label'], num_classes=6)

# Split data
X_train, X_test, y_train, y_test = train_test_split(
    padded_sequences, labels, test_size=0.2, random_state=42
)

# Build RNN model
model = Sequential([
    Embedding(5000, 64, input_length=100),
    LSTM(64, return_sequences=True),
    LSTM(32),
    Dense(32, activation='relu'),
    Dropout(0.5),
    Dense(6, activation='softmax')
])

model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

# Train model
model.fit(X_train, y_train, epochs=20, validation_data=(X_test, y_test), batch_size=32)

# Sentiment mapping
sentiment_map = {
    0: 'sadness',
    1: 'joy',
    2: 'love',
    3: 'anger',
    4: 'fear',
    5: 'surprise'
}

# Responses
def generate_response(sentiment):
    responses = {
        'sadness': [
            "I'm sorry you're feeling this way. Remember that tough times don't last forever.",
            "It's okay to feel sad sometimes. Would you like to talk about what's bothering you?",
            "I hear the sadness in your words. Be gentle with yourself today."
        ],
        'joy': [
            "It's wonderful to hear you're feeling happy! What's bringing you joy today?",
            "Your positivity is contagious! Keep spreading those good vibes.",
            "I'm so glad you're feeling joyful! Celebrate these moments."
        ],
        'love': [
            "Love is a beautiful feeling! Who or what are you feeling loving toward?",
            "The world needs more love like what you're feeling right now.",
            "Your loving energy is so powerful! Cherish these feelings."
        ],
        'anger': [
            "I sense your frustration. Would taking some deep breaths help right now?",
            "Anger can be tough to sit with. Remember it's okay to feel this way.",
            "I hear your anger. Sometimes expressing it helps release it."
        ],
        'fear': [
            "Fear can feel overwhelming. You're stronger than you think.",
            "I hear your fear. What would help you feel safer right now?",
            "Fear is a natural response. You're not alone in this feeling."
        ],
        'surprise': [
            "Wow! That sounds surprising. Would you like to share more?",
            "Surprises can be exciting or unsettling. How are you feeling about this?",
            "I didn't see that coming either! How are you processing this surprise?"
        ]
    }
    return random.choice(responses.get(sentiment, ["Thank you for sharing your feelings with me."]))

# Keyword-based sentiment override
_SENTIMENT_KEYWORDS = {
    'sadness': ['sad', 'depressed', 'unhappy', 'lonely', 'miserable', 'tear', 'cry', 'crying'],
    'joy': ['happy', 'joy', 'glad', 'excited', 'delighted', 'thrilled', 'cheerful'],
    'anger': ['angry', 'furious', 'annoyed', 'irritated', 'rage'],
    'fear': ['afraid', 'scared', 'anxious', 'anxiety', 'worried', 'terrified'],
    'love': ['love', 'loving', 'adore', 'cherish'],
    'surprise': ['surprised', 'shocked', 'astonished']
}

def predict_sentiment(user_text):
    cleaned = clean_text(user_text)
    seq = tokenizer.texts_to_sequences([cleaned])
    pad = pad_sequences(seq, maxlen=100, truncating='post')

    try:
        pred = model.predict(pad, batch_size=1, verbose=0)
    except Exception as e:
        print("Prediction error:", e)
        return 'unknown', 0.0

    label_idx = int(np.argmax(pred))
    sentiment = sentiment_map.get(label_idx, 'unknown')
    confidence = float(pred[0][label_idx])

    # Keyword override
    for key_sent, keywords in _SENTIMENT_KEYWORDS.items():
        for kw in keywords:
            pattern = r'\b' + re.escape(kw) + r'\b'
            if re.search(pattern, cleaned):
                if key_sent != sentiment:
                    sentiment = key_sent
                    confidence = max(confidence, 0.98)
                else:
                    confidence = max(confidence, 0.6)
                return sentiment, confidence

    return sentiment, confidence

# ===============================
# Conversation memory & resources
# ===============================
conversation_history = []  # fixed typo
CONTEXT_MAX_TURNS = 10

resource_library = {
    'sadness': [
        ("Understanding Depression (NIMH)", "https://www.nimh.nih.gov/health/topics/depression"),
        ("Coping with Sadness - Mind", "https://www.mind.org.uk/information-support/types-of-mental-health-problems/depression/about-depression/"),
        ("Self-care ideas when you're sad", "https://www.helpguide.org/articles/depression/coping-with-depression.htm")
    ],
    'joy': [
        ("Why gratitude increases happiness", "https://greatergood.berkeley.edu/article/item/how_gratitude_changes_you_and_your_brain"),
        ("Celebrate moments: turning joy into resilience (article)", "https://www.psychologytoday.com/us/blog/pieces-mind/201311/what-to-do-when-you-are-happily-in-love")
    ],
    'love': [
        ("The science of love (APA)", "https://www.apa.org/monitor/2015/07-08/science-love"),
        ("Nurturing relationships", "https://www.helpguide.org/articles/relationships-communication/improving-communication-in-relationships.htm")
    ],
    'anger': [
        ("Managing anger - HelpGuide", "https://www.helpguide.org/articles/relationships-communication/anger-management.htm"),
        ("Quick anger control tips - Psychology Today", "https://www.psychologytoday.com/us/blog/in-practice/201307/5-ways-cool-down-when-youre-angry")
    ],
    'fear': [
        ("Anxiety disorders (NIMH)", "https://www.nimh.nih.gov/health/topics/anxiety-disorders"),
        ("Grounding techniques for anxiety", "https://www.verywellmind.com/grounding-exercises-for-anxiety-5204203")
    ],
    'surprise': [
        ("How to cope with surprising news", "https://www.psychologytoday.com/us/blog/pieces-mind/201508/how-cope-surprising-news")
    ],
    'unknown': [
        ("Mental health basics - WHO", "https://www.who.int/news-room/fact-sheets/detail/mental-disorders")
    ]
}

def resource_html_for(sentiment):
    items = resource_library.get(sentiment, resource_library['unknown'])
    html = "<ul>"
    for title, url in items:
        html += f"<li><a href='{url}' target='_blank'>{title}</a></li>"
    html += "</ul>"
    return html

def generate_contextual_response(user_text):
    sentiment, confidence = predict_sentiment(user_text)
    base_reply = generate_response(sentiment)

    recent_user_msgs = [t[1] for t in conversation_history if t[0]=='user'][-3:]
    context_note = ""
    if recent_user_msgs:
        if len(recent_user_msgs) >= 2:
            context_note = " I remember you mentioned earlier: " + " | ".join(recent_user_msgs[-2:]) + "."
        elif len(recent_user_msgs) == 1:
            context_note = " I recall earlier you said: " + recent_user_msgs[-1] + "."

    final_reply = base_reply + context_note

    conversation_history.append(('user', user_text))
    conversation_history.append(('bot', final_reply))
    if len(conversation_history) > CONTEXT_MAX_TURNS * 2:
        conversation_history[:] = conversation_history[-CONTEXT_MAX_TURNS*2:]

    resources_html = resource_html_for(sentiment)

    ctx = conversation_history[-CONTEXT_MAX_TURNS*2:]
    summary_lines = []
    for role, txt in ctx:
        prefix = "You: " if role=='user' else "Bot: "
        display_txt = txt if len(txt) <= 120 else txt[:117] + "..."
        summary_lines.append(prefix + display_txt)
    context_summary = "\n".join(summary_lines) if summary_lines else "No context yet."

    return {
        'sentiment': sentiment,
        'confidence': confidence,
        'reply': final_reply,
        'resources_html': resources_html,
        'context_summary': context_summary
    }

def create_interactive_app_with_memory():
    text_input = widgets.Textarea(
        value='',
        placeholder='Type how you feel...',
        description='Your text:',
        layout=widgets.Layout(width='60%', height='100px')
    )
    analyze_button = widgets.Button(
        description='Analyze Sentiment', button_style='primary',
        layout=widgets.Layout(width='200px')
    )
    output_label = widgets.Label(value='Sentiment will appear here')
    response_label = widgets.Textarea(value='Response will appear here', layout=widgets.Layout(width='60%', height='80px'), disabled=True)
    context_box = widgets.Textarea(value='Context will appear here', layout=widgets.Layout(width='35%', height='300px'), disabled=True)
    resources_box = widgets.HTML(value="<i>Resources will appear here</i>", layout=widgets.Layout(width='35%', height='300px'))
    history_button = widgets.Button(description='Show Full Conversation', layout=widgets.Layout(width='200px'))

    conv_output = widgets.Output(layout=widgets.Layout(border='1px solid gray', width='95%'))

    def on_button_click(b):
        text = text_input.value
        if text.strip() == '':
            output_label.value = 'Please enter some text'
            response_label.value = ''
            return

        res = generate_contextual_response(text)
        output_label.value = f"Detected sentiment: {res['sentiment']} (confidence: {res['confidence']:.2f})"
        response_label.value = res['reply']
        context_box.value = res['context_summary']
        resources_box.value = f"<h4>Resources for {res['sentiment'].capitalize()}</h4>" + res['resources_html']
        text_input.value = ''

    def on_history(b):
        with conv_output:
            conv_output.clear_output()
            print("=== Full conversation (session) ===")
            for role, txt in conversation_history:
                prefix = "You: " if role=='user' else "Bot: "
                print(prefix + txt)

    analyze_button.on_click(on_button_click)
    history_button.on_click(on_history)

    left_col = widgets.VBox([
        widgets.HTML("<h2>Sentiment Analysis Chat (with memory)</h2>"),
        widgets.HTML("<p>Share how you're feeling, and I'll respond accordingly. I will remember this session's conversation.</p>"),
        text_input,
        widgets.HBox([analyze_button, history_button]),
        widgets.HTML("<h3>Bot Reply</h3>"),
        response_label,
        conv_output
    ], layout=widgets.Layout(width='62%'))

    right_col = widgets.VBox([
        widgets.HTML("<h3>Quick Info</h3>"),
        output_label,
        widgets.HTML("<h3>Context Summary</h3>"),
        context_box,
        widgets.HTML("<h3>Resource Library</h3>"),
        resources_box
    ], layout=widgets.Layout(width='36%'))

    display(widgets.HBox([left_col, right_col]))

# ===============================
# Journal feature
# ===============================
def journal_mode():
    entry_box = widgets.Textarea(
        value='', placeholder='Write your journal entry...',
        description='Journal:',
        layout=widgets.Layout(width='80%', height='150px')
    )
    save_button = widgets.Button(description="Save Entry", button_style='success')
    status_label = widgets.Label(value='')

    def save_entry(b):
        entry = entry_box.value.strip()
        if entry == '':
            status_label.value = "Please write something before saving."
            return
        file_exists = os.path.exists("journal.csv")
        df_entry = pd.DataFrame([{
            "date": datetime.now().strftime("%Y-%m-%d %H:%M:%S"),
            "entry": entry
        }])
        df_entry.to_csv("journal.csv", mode='a', header=not file_exists, index=False)
        status_label.value = "Journal entry saved!"
        entry_box.value = ''

    save_button.on_click(save_entry)

    display(widgets.VBox([
        widgets.HTML("<h2>📝 Journal Mode</h2>"), entry_box, save_button, status_label
    ]))

# ============================================================
# Mental Health Assessment
# ============================================================
def mental_health_assessment_mode():
    # Imports specific to this tool (kept local)
    from sklearn.linear_model import LogisticRegression
    from sklearn.metrics import accuracy_score
    import matplotlib.pyplot as plt
    import seaborn as sns  # optional; included as in your snippet

    np.random.seed(42)

    def create_synthetic_data(n_samples=1000):
        data = {
            'Age': np.random.randint(18, 70, size=n_samples),
            'Gender': np.random.choice(['Male', 'Female', 'Other'], size=n_samples),
            'Sleep_Hours': np.random.normal(7, 1.5, n_samples).clip(3, 12),
            'Exercise_Days': np.random.randint(0, 7, size=n_samples),
            'Social_Interaction': np.random.randint(1, 10, size=n_samples),
            'Stress_Level': np.random.randint(1, 10, size=n_samples),
            'Mood': np.random.randint(1, 10, size=n_samples),
            'Productivity': np.random.randint(1, 10, size=n_samples),
            'Has_Therapist': np.random.choice([0, 1], size=n_samples),
        }

        depression_risk = (
            0.1 * data['Age'] +
            -0.5 * data['Sleep_Hours'] +
            -0.3 * data['Exercise_Days'] +
            -0.4 * data['Social_Interaction'] +
            0.8 * data['Stress_Level'] +
            -0.7 * data['Mood'] +
            -0.2 * data['Productivity'] +
            0.4 * data['Has_Therapist'] * 10 +
            np.random.normal(0, 2, n_samples)
        )
        data['Depression_Risk'] = (depression_risk > depression_risk.mean()).astype(int)
        return pd.DataFrame(data)

    # Generate and preprocess
    mh_df = create_synthetic_data(1000)
    mh_df = pd.get_dummies(mh_df, columns=['Gender'], drop_first=True)

    X_mh = mh_df.drop('Depression_Risk', axis=1)
    y_mh = mh_df['Depression_Risk']

    X_train_mh, X_test_mh, y_train_mh, y_test_mh = train_test_split(
        X_mh, y_mh, test_size=0.2, random_state=42
    )

    # Train model
    mh_model = LogisticRegression(max_iter=1000)
    mh_model.fit(X_train_mh, y_train_mh)

    # (Optional) quick feedback to console
    train_acc = (mh_model.predict(X_train_mh) == y_train_mh).mean()
    test_acc = (mh_model.predict(X_test_mh) == y_test_mh).mean()

    # Widgets
    age_slider = widgets.IntSlider(value=30, min=18, max=70, step=1, description='Age:', continuous_update=False)
    gender_dropdown = widgets.Dropdown(options=['Male', 'Female', 'Other'], value='Male', description='Gender:')
    sleep_slider = widgets.FloatSlider(value=7, min=3, max=12, step=0.5, description='Sleep Hours:', continuous_update=False)
    exercise_slider = widgets.IntSlider(value=3, min=0, max=7, step=1, description='Exercise Days:', continuous_update=False)
    social_slider = widgets.IntSlider(value=5, min=1, max=10, step=1, description='Social Interaction:', continuous_update=False)
    stress_slider = widgets.IntSlider(value=5, min=1, max=10, step=1, description='Stress Level:', continuous_update=False)
    mood_slider = widgets.IntSlider(value=5, min=1, max=10, step=1, description='Mood:', continuous_update=False)
    productivity_slider = widgets.IntSlider(value=5, min=1, max=10, step=1, description='Productivity:', continuous_update=False)
    therapist_toggle = widgets.ToggleButtons(options=[('No', 0), ('Yes', 1)], description='Has Therapist:')

    predict_button = widgets.Button(description='Assess Mental Health', button_style='info', tooltip='Click to predict depression risk')
    mh_output = widgets.Output()

    expected_columns = X_train_mh.columns

    def predict_depression_risk(input_data):
        input_df = pd.DataFrame([input_data])
        for col in expected_columns:
            if col not in input_df.columns:
                input_df[col] = 0
        input_df = input_df[expected_columns]
        probability = mh_model.predict_proba(input_df)[0][1]
        prediction = mh_model.predict(input_df)[0]
        return probability, prediction

    def on_predict_button_clicked(b):
        with mh_output:
            clear_output()

            input_data = {
                'Age': age_slider.value,
                'Sleep_Hours': sleep_slider.value,
                'Exercise_Days': exercise_slider.value,
                'Social_Interaction': social_slider.value,
                'Stress_Level': stress_slider.value,
                'Mood': mood_slider.value,
                'Productivity': productivity_slider.value,
                'Has_Therapist': therapist_toggle.value,
                'Gender_Female': 1 if gender_dropdown.value == 'Female' else 0,
                'Gender_Other': 1 if gender_dropdown.value == 'Other' else 0
            }

            probability, prediction = predict_depression_risk(input_data)

            print("\n=== Mental Health Assessment Results ===")
            print(f"Model train/test accuracy: {train_acc:.2f} / {test_acc:.2f}")
            print(f"Probability of depression risk: {probability:.1%}")

            if probability < 0.3:
                print("Interpretation: Low risk of depression")
                print("Suggestions: Maintain your healthy habits!")
            elif probability < 0.6:
                print("Interpretation: Moderate risk of depression")
                print("Suggestions: Consider stress management techniques and social support")
            else:
                print("Interpretation: High risk of depression")
                print("Suggestions: Please consider seeking professional help")

            # Visualization
            import matplotlib.pyplot as plt
            plt.figure(figsize=(8, 2))
            # Using default colors; you can style if you like
            plt.barh(['Depression Risk'], [probability])
            plt.xlim(0, 1)
            plt.title('Depression Risk Probability')
            plt.show()

    predict_button.on_click(on_predict_button_clicked)

    # Layout
    display(widgets.VBox([
        widgets.HTML("<h2>Mental Health Assessment Tool</h2>"),
        widgets.HTML("<p>Please answer the following questions about your mental health:</p>"),
        age_slider,
        gender_dropdown,
        sleep_slider,
        exercise_slider,
        social_slider,
        stress_slider,
        mood_slider,
        productivity_slider,
        therapist_toggle,
        predict_button,
        mh_output,
        widgets.HTML("""
        <div style="margin-top: 20px; padding: 10px; background-color: #f0f0f0; border-radius: 5px;">
            <h4>About This Tool</h4>
            <p>This is a basic demonstration of a mental health prediction tool using machine learning.
            The model was trained on synthetic data and is for educational purposes only.</p>
            <p><strong>Note:</strong> This is not a diagnostic tool. If you have concerns about your mental health,
            please consult a qualified healthcare professional.</p>
        </div>
        """)
    ]))

# ===============================
# Main menu
# ===============================
def main_menu():
    mh_button = widgets.Button(description="🧠 Mental Health Assessment", button_style='warning')
    bot_button = widgets.Button(description="🤖 Talk to Bot", button_style='primary')
    journal_button = widgets.Button(description="📓 Journal", button_style='info')

    def open_mh(b):
        display(widgets.HTML("<h3>Mental Health Assessment</h3>"))
        mental_health_assessment_mode()

    def open_bot(b):
        display(widgets.HTML("<h3>Bot Mode</h3>"))
        create_interactive_app_with_memory()

    def open_journal(b):
        display(widgets.HTML("<h3>Journal Mode</h3>"))
        journal_mode()

    bot_button.on_click(open_bot)
    journal_button.on_click(open_journal)
    mh_button.on_click(open_mh)

    display(widgets.VBox([
        widgets.HTML("<h2>Welcome! Choose a mode:</h2>"),
        widgets.HBox([mh_button, bot_button, journal_button])
    ]))

# Run the menu
main_menu()


/usr/local/lib/python3.11/dist-packages/keras/src/layers/core/embedding.py:97: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


Epoch 1/20
50/50 ━━━━━━━━━━━━━━━━━━━━ 14s 40ms/step - accuracy: 0.3039 - loss: 1.7243 - val_accuracy: 0.3275 - val_loss: 1.6011
Epoch 2/20
50/50 ━━━━━━━━━━━━━━━━━━━━ 1s 16ms/step - accuracy: 0.3145 - loss: 1.6356 - val_accuracy: 0.3275 - val_loss: 1.5723
Epoch 3/20
50/50 ━━━━━━━━━━━━━━━━━━━━ 1s 19ms/step - accuracy: 0.3189 - loss: 1.5712 - val_accuracy: 0.3450 - val_loss: 1.5238
Epoch 4/20
50/50 ━━━━━━━━━━━━━━━━━━━━ 1s 20ms/step - accuracy: 0.4367 - loss: 1.3083 - val_accuracy: 0.4200 - val_loss: 1.4654
Epoch 5/20
50/50 ━━━━━━━━━━━━━━━━━━━━ 1s 13ms/step - accuracy: 0.5436 - loss: 1.0749 - val_accuracy: 0.5000 - val_loss: 1.4396
Epoch 6/20
50/50 ━━━━━━━━━━━━━━━━━━━━ 1s 13ms/step - accuracy: 0.6915 - loss: 0.8122 - val_accuracy: 0.5100 - val_loss: 1.4358
Epoch 7/20
50/50 ━━━━━━━━━━━━━━━━━━━━ 1s 13ms/step - accuracy: 0.7604 - loss: 0.6527 - val_accuracy: 0.5500 - val_loss: 1.6013
Epoch 8/20
50/50 ━━━━━━━━━━━━━━━━━━━━ 1s 14ms/step - accuracy: 0.7757 - loss: 0.6119 - val_accuracy: 0.5475 - 

HTML(value='<h3>Journal Mode</h3>')

HTML(value='<h3>Bot Mode</h3>')

HTML(value='<h3>Mental Health Assessment</h3>')